# Downloading necessary libraries

In [1]:
!pip -qq install torch torchvision torchaudio
!pip -qq install matplotlib pandas numpy scikit-learn openpyxl

# Import necessary libraries

In [2]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

# Device agnostic

In [3]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

# Load the data

In [4]:
data = pd.read_excel("data/openings.xlsx")
data.head()

,space,room_size,capacity,user_per_min,no_of_openings,width1,width2,width3,width4,width5
0,Area for standing,50,167,5,1,0.65,0.0,0.0,0.0,0.0
1,Area for standing,75,250,5,1,0.65,0.0,0.0,0.0,0.0
2,Area for standing,100,333,5,1,0.65,0.0,0.0,0.0,0.0
3,Area for standing,125,417,5,1,0.65,0.0,0.0,0.0,0.0
4,Area for standing,150,500,5,1,0.65,0.0,0.0,0.0,0.0


In [6]:
X = data.drop(['no_of_openings', 'width1', 'width2', 'width3', 'width4', 'width5'], axis=1)

In [7]:
y = data.drop(["space", "room_size", "capacity", "user_per_min"], axis=1)

## Split the data into train and test data

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Data Preprocessing

In [9]:
num_attribs = ["room_size", "capacity", "user_per_min"]
cat_attribs = ["space"]

num_pipeline = Pipeline([('std_scaler', StandardScaler())])

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

In [10]:
X_train_tensor = torch.Tensor(full_pipeline.fit_transform(X_train)).to(device)
X_test_tensor = torch.Tensor(full_pipeline.transform(X_test)).to(device)
y_train_tensor = torch.from_numpy(y_train.values).float().to(device)
y_test_tensor = torch.from_numpy(y_test.values).float().to(device)

In [18]:
X_train_tensor.shape

torch.Size([6615, 13])

# Building the model

In [31]:
class OpeningsPredictor(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_stack = nn.Sequential(
            nn.Linear(in_features=13, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=6)
        )

    def forward(self, X):
        return self.layer_stack(X)

In [32]:
openings_predictor = OpeningsPredictor().to(device)

In [33]:
loss_fn = nn.L1Loss()
optimizer = torch.optim.Adam(params=openings_predictor.parameters(), lr=0.1)

# Training the model

In [41]:
torch.manual_seed(42)
epochs = 1000

epoch_count = []
train_loss_values = []
test_loss_values = []

for epoch in range(epochs):

    # Training
    openings_predictor.train()
    train_predictions = openings_predictor(X_train_tensor)
    train_loss = loss_fn(train_predictions, y_train_tensor)
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    # Testing
    openings_predictor.eval()
    with torch.inference_mode():
        test_predictions = openings_predictor(X_test_tensor)
        test_loss = loss_fn(test_predictions, y_test_tensor)

    # Print out what's happening
    if epoch % 100 == 0:
        epoch_count.append(epoch)
        train_loss_values.append(train_loss)
        test_loss_values.append(test_loss)
        print(f"Epoch: {epoch} | Loss: {train_loss:.5f} | Test Loss: {test_loss:.5f}")

AttributeError: 'int' object has no attribute 'cpu'

# Plot the loss

In [40]:
plt.plot(epoch_count, train_loss_values, label="Train loss")
plt.plot(epoch_count, test_loss_values, label="Test loss")
plt.title("Training and test loss curves")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend()

AttributeError: 'list' object has no attribute 'cpu'

# Saving the model

In [ ]:
torch.save(openings_predictor, "../models/openings.pth")